In [54]:
import matplotlib.pyplot as plt
from collections import OrderedDict
%matplotlib inline


def users_ingredient_score(user_id):
    ing_dict = {}    
    for fc in UserFlavorCompound.objects.filter(user_id=user_id):
        ingredient_list = IngredientFlavorCompound.objects.filter(flavor_id=fc.id)
        
        for ingredient in ingredient_list:
            if ingredient.ingredient_id in ing_dict.keys():
                ing_dict[ingredient.ingredient_id] += fc.score
            else:
                ing_dict[ingredient.ingredient_id] = fc.score
    return OrderedDict(sorted(ing_dict.items(), key=lambda t: t[1]))

def perfect_ingr(user_id):
    fc_list = set(UserFlavorCompound.objects.filter(user_id=user_id, score__gt=0).values_list('flavor_id', flat=True))
    unique_ingr_list = IngredientFlavorCompound.objects.distinct().values_list('ingredient_id', flat=True)
    full_hit_list = []
    
    for ingredient in unique_ingr_list:
        ing_fc_list = set(IngredientFlavorCompound.objects.filter(ingredient_id=ingredient).values_list('flavor_id', flat=True))
        match = set.intersection(fc_list, ing_fc_list)
        hit_percent = len(match) / len(ing_fc_list)
        full_hit_list.append((ingredient, hit_percent))
    return sorted(full_hit_list, key=lambda x: x[1], reverse=True)

In [ ]:
perfect_ingr(2)